# Transformers kütüphanesi kullanılarak CLIPTextModel ve CLIPTokenizer modelleri oluşturulur. 

    1) Tokenizer modeli kullanılarak verilen metinler tokenize edilir. 
    2) TextModel modeli kullanılarak tokenize edilen metinlerin görsel embedding vektörleri elde edilir.

In [ ]:
from diffusers import StableDiffusionPipeline

In [ ]:
import torch

from PIL import Image
from transformers import CLIPTextModel, CLIPTokenizer

In [ ]:
# Negative prompts embed eklenmedi, prompt embed test edilince eklenecek.
def encode_prompt(tokenizer, text_encoder, prompt, device, negative_prompt=None):
   
    # Prompt tokenize
    tokens = tokenizer(prompt, 
                       padding=False,
                       truncation=False,
                       return_tensors="pt")
    
    print("Tokens:", tokens, "attention_mask hakkinda daha fazla bilgi icin: https://huggingface.co/transformers/glossary.html#attention-mask")
        
    tokens_ids = tokens.input_ids
    print("Tokens IDs:", tokens_ids)

    # to device
    tokens_ids = tokens_ids.to(device)
    text_encoder = text_encoder.eval().to(device)


    # Prompt embeds
    prompt_embeds = text_encoder(tokens_ids)
    prompt_embeds = prompt_embeds[0]
    print("Prompt Embeds:", prompt_embeds)

    return prompt_embeds

In [ ]:
# Prompt Tokenize ve Embedding Test
prompt = "A photo of a cat"
device = "cuda" if torch.cuda.is_available() else "cpu"

text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

embeds = encode_prompt(tokenizer, text_encoder, prompt, device)
print("Embeds:", embeds)

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", safety_checker=None)
pipe = pipe.to("cuda", torch.float16)

generator = torch.Generator(device="cuda").manual_seed(50)
generated = pipe(prompt_embeds=embeds,
                generator=generator,
                num_inference_steps = 20,
                height = 512,
                width = 512,            
                guidance_scale = 8).images[0]

display(generated)